In [1]:
import pandas as pd
import sqlite3

In [2]:
pd.options.display.max_columns = None

In [3]:
conn = sqlite3.connect('sakila.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
SELECT * FROM sqlite_master WHERE type = 'table'
''')
data = cursor.fetchall()
columns = [col[0] for col in cursor.description]
cursor.close()

In [5]:
data_df = pd.DataFrame(data, columns=columns)
data_df

,type,name,tbl_name,rootpage,sql
0,table,actor,actor,2,"CREATE TABLE ""actor"" (\n""index"" INTEGER,\n ""a..."
1,table,store,store,7,"CREATE TABLE ""store"" (\n""index"" INTEGER,\n ""s..."
2,table,address,address,9,"CREATE TABLE ""address"" (\n""index"" INTEGER,\n ..."
3,table,category,category,26,"CREATE TABLE ""category"" (\n""index"" INTEGER,\n ..."
4,table,city,city,28,"CREATE TABLE ""city"" (\n""index"" INTEGER,\n ""ci..."
5,table,country,country,39,"CREATE TABLE ""country"" (\n""index"" INTEGER,\n ..."
6,table,customer,customer,43,"CREATE TABLE ""customer"" (\n""index"" INTEGER,\n ..."
7,table,film_actor,film_actor,63,"CREATE TABLE ""film_actor"" (\n""index"" INTEGER,\..."
8,table,film_category,film_category,126,"CREATE TABLE ""film_category"" (\n""index"" INTEGE..."
9,table,inventory,inventory,140,"CREATE TABLE ""inventory"" (\n""index"" INTEGER,\n..."


Task 1:

Write a query that displays a table with 4 columns: actor's full name, film title, length of movie, and a column name "filmlen_groups" that classifies movies based on their length. Filmlen_groups should include 4 categories: 1 hour or less, Between 1-2 hours, Between 2-3 hours, More than 3 hours.

Match the filmlen_groups with the movie titles in your result dataset.

In [13]:
pd.read_sql_query(sql = '''
SELECT first_name || ' ' || last_name AS full_name, title, length, 
CASE WHEN length <= 60 THEN '1 hour or less' WHEN length > 60 AND length <= 120 THEN 'Between 1-2 hours'
WHEN length > 120 AND length <= 180 THEN 'Between 2-3 hours' ELSE 'More than 3 hours' END AS filmlen_groups
FROM actor a
JOIN film_actor fa
ON a.actor_id = fa.actor_id
JOIN film f
ON fa.film_id = f.film_id;
''', con = conn)

,full_name,title,length,filmlen_groups
0,Penelope Guiness,Academy Dinosaur,86,Between 1-2 hours
1,Penelope Guiness,Anaconda Confessions,92,Between 1-2 hours
2,Penelope Guiness,Angels Life,74,Between 1-2 hours
3,Penelope Guiness,Bulworth Commandments,61,Between 1-2 hours
4,Penelope Guiness,Cheaper Clyde,87,Between 1-2 hours
...,...,...,...,...
5457,Thora Temple,Telegraph Voyage,148,Between 2-3 hours
5458,Thora Temple,Trojan Tomorrow,52,1 hour or less
5459,Thora Temple,Virginian Pluto,164,Between 2-3 hours
5460,Thora Temple,Wardrobe Phantom,178,Between 2-3 hours


In [14]:
pd.read_sql_query(sql = '''
SELECT first_name || ' ' || last_name AS full_name, title, length, 
CASE WHEN length <= 60 THEN '1 hour or less' WHEN length > 60 AND length <= 120 THEN 'Between 1-2 hours'
WHEN length > 120 AND length <= 180 THEN 'Between 2-3 hours' ELSE 'More than 3 hours' END AS filmlen_groups
FROM actor a
JOIN film_actor fa
ON a.actor_id = fa.actor_id
JOIN film f
ON fa.film_id = f.film_id
WHERE title = 'Academy Dinosaur' OR title = 'Color Philadelphia' OR title = 'Oklahoma Jumanji';
''', con = conn)

,full_name,title,length,filmlen_groups
0,Penelope Guiness,Academy Dinosaur,86,Between 1-2 hours
1,Christian Gable,Academy Dinosaur,86,Between 1-2 hours
2,Lucille Tracy,Academy Dinosaur,86,Between 1-2 hours
3,Sandra Peck,Academy Dinosaur,86,Between 1-2 hours
4,Johnny Cage,Academy Dinosaur,86,Between 1-2 hours
5,Mena Temple,Academy Dinosaur,86,Between 1-2 hours
6,Warren Nolte,Academy Dinosaur,86,Between 1-2 hours
7,Oprah Kilmer,Academy Dinosaur,86,Between 1-2 hours
8,Rock Dukakis,Academy Dinosaur,86,Between 1-2 hours
9,Mary Keitel,Academy Dinosaur,86,Between 1-2 hours


Solution:

SELECT full_name, 
       filmtitle,
       filmlen,
       CASE WHEN filmlen <= 60 THEN '1 hour or less'
       WHEN filmlen > 60 AND filmlen <= 120 THEN 'Between 1-2 hours'
       WHEN filmlen > 120 AND filmlen <= 180 THEN 'Between 2-3 hours'
       ELSE 'More than 3 hours' END AS filmlen_groups
FROM 
    (SELECT a.first_name, 
               a.last_name,
               a.first_name || ' ' || a.last_name AS full_name,
               f.title filmtitle, 
               f.length filmlen
        FROM film_actor fa
        JOIN actor a
        ON fa.actor_id = a.actor_id
        JOIN film f
        ON f.film_id = fa.film_id) t1

Task 2:

Now, we bring in the advanced SQL query concepts! Revise the query you wrote above to create a count of movies in each of the 4 filmlen_groups: 1 hour or less, Between 1-2 hours, Between 2-3 hours, More than 3 hours.

Match the count of movies in each filmlen_group.

In [16]:
pd.read_sql_query(sql = '''
SELECT 
CASE WHEN length <= 60 THEN '1 hour or less' WHEN length > 60 AND length <= 120 THEN 'Between 1-2 hours'
WHEN length > 120 AND length <= 180 THEN 'Between 2-3 hours' ELSE 'More than 3 hours' END AS filmlen_groups,
COUNT(*) count
FROM film
GROUP BY 1
ORDER BY 1;
''', con = conn)

,filmlen_groups,count
0,1 hour or less,104
1,Between 1-2 hours,439
2,Between 2-3 hours,418
3,More than 3 hours,39


Solution:

SELECT    DISTINCT(filmlen_groups),
          COUNT(title) OVER (PARTITION BY filmlen_groups) AS filmcount_bylencat
FROM  
         (SELECT title,length,
          CASE WHEN length <= 60 THEN '1 hour or less'
          WHEN length > 60 AND length <= 120 THEN 'Between 1-2 hours'
          WHEN length > 120 AND length <= 180 THEN 'Between 2-3 hours'
          ELSE 'More than 3 hours' END AS filmlen_groups
          FROM film ) t1
ORDER BY  filmlen_groups